In [1]:
import numpy as np
import pandas as pd

In [10]:
match_info = ["Season",
              "Div",
              "Date",
              "HomeTeam",
              "AwayTeam"]
match_statistics = ["FTHG",
                    "FTAG",
                    "FTR",
                    "HTHG",
                    "HTAG",
                    "HTR",
                    "Attendance",
                    "Referee",
                    "HS",
                    "AS",
                    "HST",
                    "AST",
                    "HHW",
                    "AHW",
                    "HC",
                    "AC",
                    "HF",
                    "AF",
                    "HO",
                    "AO",
                    "HY",
                    "AY",
                    "HR",
                    "AR",
                    "HBP",
                    "ABP"]
hs_dtypes = {"Season": np.int64,
            "Div": object,
            "Date": np.datetime64,
            "HomeTeam": object,
            "AwayTeam": object,
            "FTHG": np.int64,
            "FTAG": np.int64,
            "FTR": np.int64,
            "HTHG": np.int64,
            "HTAG": np.int64,
            "HTR": np.int64,
            "Attendance": np.int64,
            "Referee": object,
            "HS": np.int64,
            "AS": np.int64,
            "HST": np.int64,
            "AST": np.int64,
            "HHW": np.int64,
            "AHW": np.int64,
            "HC": np.int64,
            "AC": np.int64,
            "HF": np.int64,
            "AF": np.int64,
            "HO": np.int64,
            "AO": np.int64,
            "HY": np.int64,
            "AY": np.int64,
            "HR": np.int64,
            "AR": np.int64,
            "HBP": np.int64,
            "ABP": np.int64}

In [13]:
historical_scores = pd.read_csv("../data/processed/historical_scores.csv", index_col=0, dtype=hs_dtypes, na_values="NaN")
historical_scores = historical_scores[match_info + match_statistics]
measures = historical_scores[match_info]

Exception: Integer column has NA values in column 1

In [19]:
historical_scores[pd.isnull(historical_scores.FTHG)]

,Season,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,...,HF,AF,HO,AO,HY,AY,HR,AR,HBP,ABP
462,9394,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
463,9394,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
464,9394,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
465,9394,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
466,9394,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
467,9394,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
468,9394,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
469,9394,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
470,9394,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
471,9394,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
historical_scores.dtypes

Season          int64
Div            object
Date           object
HomeTeam       object
AwayTeam       object
FTHG          float64
FTAG          float64
FTR            object
HTHG          float64
HTAG          float64
HTR            object
Attendance    float64
Referee        object
HS            float64
AS            float64
HST           float64
AST           float64
HHW           float64
AHW           float64
HC            float64
AC            float64
HF            float64
AF            float64
HO            float64
AO            float64
HY            float64
AY            float64
HR            float64
AR            float64
HBP           float64
ABP           float64
dtype: object

In [4]:
def get_match_information(df):
    

Season       int64
Div         object
Date        object
HomeTeam    object
AwayTeam    object
dtype: object